### initiate amazon s3 connection

In [ ]:
# Let's use Amazon S3

import boto3
import json
import pandas as pd

s3c = boto3.client('s3')
s3r = boto3.resource('s3')

### load all buckets from s3 and second bucket's objects

In [ ]:
buckets = []

# Print out bucket names
for bucket in s3r.buckets.all():
    buckets.append(bucket)

In [ ]:
for i in buckets[1].objects.iterator():
    pass
    #print(i)

In [ ]:
documents = s3c.list_objects(Bucket = buckets[1].name)['Contents']
textract_c = boto3.client('textract')

documents_ls = [d['Key'] for d in documents]
#documents_ls

### get responses and extract their contents from amazon textract

In [ ]:
response_ls = []
for d in documents_ls:
    response_ls.append(textract_c.start_document_analysis(DocumentLocation={'S3Object':{'Bucket':buckets[1].name,'Name':d}},FeatureTypes = ['TABLES']))
    
jobId_ls = [r['JobId'] for r in response_ls]

In [ ]:
result_ls = []

for j in jobId_ls:
    result_ls.append(textract_c.get_document_analysis(JobId=j))

In [ ]:
jobStatus = [r['JobStatus'] for r in result_ls]
jobStatus_df = pd.DataSeries(jobStatus)

### Saving each raw text in result_ls to .json

In [ ]:
file_head = 'city of san jose minutes_'
file_surfix = '.json'
i = 0

for r in result_ls:
    raw_txt,raw_ls = creatRaw(result_ls[i])
    with open(file_head+str(i+1)+file_surfix,'w') as f:
        json.dump(result_ls[i],f)
        f.close()
    i+=1
    

### create raw text and raw content list from textract string input.

In [ ]:
def creatRaw(result):

    raw_ls=[]
    raw_txt=''
    for t in result['Blocks']:
        try:
            if t['Text'][:4]=='Page':
                pass
            else:
                raw_ls.append( t['Text'])
                raw_txt += t['Text']+' '
        except:
            pass

    return raw_txt,raw_ls

### load and create raw text file from .json file.

In [ ]:
import json

with open('./raw/city of san jose minutes_19.json') as jsonFile:
    data = json.load(jsonFile)
    
rawtxt, rawls = creatRaw(data)